In [2]:
import random
import numpy  as np

In [40]:
""" # generate a random matrix

# 设置种子
np.random.seed(10)
# 生成矩阵
mt1 = np.random.randint(0, 100, size=250)
""" splits = np.array_split(mt1, 50)
print(splits)
"""
def extract_basics_split(data):
    means = np.empty((1, 50))
    stds = np.empty((1, 50))
    maxs = np.empty((1, 50))
    mins = np.empty((1, 50))
    medians = np.empty((1, 50))

    splits = np.array_split(data, 50)
    
    for j in range(len(splits)):
        means[0, j] = np.mean(splits[j])
        stds[0, j] = np.std(splits[j])
        maxs[0, j] = np.max(splits[j])
        mins[0, j] = np.min(splits[j])
        medians[0, j] = np.median(splits[j])

    features = np.array([means, stds, maxs, mins, medians])

    features = features.T
    return features

features = extract_basics_split(mt1)
print(features) """

[[[41.         30.66594202 89.          9.         28.        ]]

 [[40.6        36.4450271  93.          0.         29.        ]]

 [[31.4        15.86946754 54.         11.         36.        ]]

 [[66.6        18.8        88.         33.         72.        ]]

 [[60.         11.027239   77.         49.         54.        ]]

 [[45.8        35.59438158 92.         13.         25.        ]]

 [[45.2        27.83810338 89.         12.         30.        ]]

 [[33.8        13.01383879 57.         18.         31.        ]]

 [[61.8        32.6521056  94.         22.         77.        ]]

 [[42.         32.82072516 88.          9.         28.        ]]

 [[54.4        31.42355804 88.         15.         71.        ]]

 [[31.2        25.8178233  75.          7.         17.        ]]

 [[65.8        29.12318664 96.         28.         84.        ]]

 [[42.4        33.97999411 88.          4.         44.        ]]

 [[48.2        24.18594633 88.         15.         50.        ]]

 [[54.2   

# Load in the data

In [59]:
#Import libaries
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, ifft
from scipy.io import wavfile 
from scipy.signal import filtfilt, butter
from pylab import *
import os
import librosa

#Import self-written functions
from Denoise_Functions import *
from features_functions import *

#Get directories with data
dir_gearbox = os.path.join(os.getcwd(), 'gearbox')
dir_train = os.path.join(dir_gearbox, "train")
dir_test = os.path.join(dir_gearbox, "test")

# Extract anomoly files and normal files from the test set
anom_files = []
anom_sr = []
anom_data = []
anom_data_fft = []

norm_files = []
norm_sr = []
norm_data = []
norm_data_fft = []

# make a list of all the files in the training data folder that contain section_00 in the name and volt_1.0 in the name
for i in os.listdir(dir_test):
    if 'anomaly' in i and 'section_00' in i and 'volt_1.0' in i:
        anom_files.append(i)
        # data, samplerate = librosa.load(dir_test + '/' + i,sr=16000)
        samplerate, data = wavfile.read(dir_test + '\\' + i)
        anom_sr.append(samplerate)
        anom_data.append(data)

        #Also store the fft data
        data_fft_v1 = fft(data)
        data_fft = 2.0/(data.shape[0]) * np.abs(data_fft_v1[0:data.shape[0]//2])
        anom_data_fft.append(data_fft)

    if 'normal' in i and 'section_00' in i and 'volt_1.0' in i:
        norm_files.append(i)
        # data, samplerate = librosa.load(dir_test + '/' + i,sr=16000)
        samplerate, data = wavfile.read(dir_test + '\\' + i)
        norm_sr.append(samplerate)
        norm_data.append(data)

        #Also store the fft data
        data_fft_v1 = fft(data)
        data_fft = 2.0/(data.shape[0]) * np.abs(data_fft_v1[0:data.shape[0]//2])
        norm_data_fft.append(data_fft)

# extract basic features
anom_means, anom_stds, anom_maxs, anom_mins, anom_medians = extract_basics_split(anom_data)
norm_means, norm_stds, norm_maxs, norm_mins, norm_medians = extract_basics_split(norm_data)

# convert list objectives to numpy array form
anom_means_arr = np.array(anom_means)
anom_stds_arr = np.array(anom_stds)
anom_maxs_arr = np.array(anom_maxs)
anom_mins_arr = np.array(anom_mins)
anom_medians_arr = np.array(anom_medians)
norm_means_arr = np.array(norm_means)
norm_stds_arr = np.array(norm_stds)
norm_maxs_arr = np.array(norm_maxs)
norm_mins_arr = np.array(norm_mins)
norm_medians_arr = np.array(norm_medians)

# create a matrix of size n_samples times n_features
X_anom = np.stack([anom_means_arr, anom_stds_arr, anom_maxs_arr, anom_mins_arr, 
                         anom_medians_arr], axis=0).T

X_norm = np.stack([norm_means_arr, norm_stds_arr, norm_maxs_arr, 
                         norm_mins_arr, norm_medians_arr], axis=0).T



(500, 5)


In [70]:
import random
import numpy  as np

# define the fitness function
def  calculate_fitness(individual , X):
    # Convert  binary  string  to  boolean  mask
    mask = np.array(list(individual), dtype=bool)
    # Apply  mask to  dataset
    X_masked = X[:, mask]
    # Calculate  variance  of each  feature
    variances = np.var(X_masked , axis =0)
    # Calculate  fitness  as the  sum of  variances
    fitness = np.sum(variances)
    
    return  fitness


# define the genetic algorithm
def genetic_algorithm(data, population_size, num_genes, calculate_fitness, mutation_rate, elitism_rate, num_generations):
    # initialize the population
    population = []
    for i in range(population_size):
        individual = [random.randint(0, 1) for j in range(num_genes)]
        population.append(individual)
        # print(population)

    # run the evolution loop for num_generations
    for generation in range(num_generations):
        # evaluate the fitness of each individual in the population
        fitness_values = [calculate_fitness(individual, data) for individual in population]

        # select the fittest individuals for the next generation
        num_elites = int(elitism_rate * population_size)
        elites = sorted(range(len(population)), key=lambda i: fitness_values[i], reverse=True)[:num_elites]
        next_generation = [population[i] for i in elites]

        # breed new individuals to fill the rest of the next generation
        while len(next_generation) < population_size:
            parent1, parent2 = random.choices(population, weights=fitness_values, k=2)
            child = [parent1[i] if random.random() < 0.5 else parent2[i] for i in range(num_genes)]
            if random.random() < mutation_rate:
                gene_to_mutate = random.randint(0, num_genes-1)
                child[gene_to_mutate] = 1 - child[gene_to_mutate]
            next_generation.append(child)

        # replace the old population with the new generation
        population = next_generation
        # print(population)

    # return the fittest individual found
    return max(population, key=lambda individual: calculate_fitness(individual, data))

data = X_anom
population_size = 32   # 2 to the n_feature th, which means all the combination of features
num_genes = 5
mutation_rate=0.01
elitism_rate=0.1
num_generations=5

features = genetic_algorithm(data, population_size, num_genes, calculate_fitness, mutation_rate=0.01, elitism_rate=0.1, num_generations=100)
print(features)

[1, 1, 1, 1, 1]
